This demo program solves the out-of-plane Kirchhoff-Love equations on the unit.

Square with uniform transverse loading and fully clamped boundary conditions.

In [58]:
from dolfin import *
from fenics_shells import *

# and creating a mesh::

mesh = UnitSquareMesh(256, 256)

In [59]:
element_W = FiniteElement("Lagrange", triangle,  2)
W = FunctionSpace(mesh, element_W)

In [60]:
w_ = Function(W)
w = TrialFunction(W)
w_t = TestFunction(W)

In [61]:
E = Constant(10920)
nu = Constant(0.3)
t = Constant(0.001)

In [62]:
theta = grad(w_)

In [63]:
k = variable(sym(grad(theta)))

In [64]:
D = (E*t**3)/(24.0*(1.0 - nu**2))
psi_M = D*((1.0 - nu)*tr(k*k) + nu*(tr(k))**2)

In [65]:
M = diff(psi_M, k)

In [66]:
alpha = E*t**3
h = CellDiameter(mesh)
h_avg = (h('+') + h('-'))/2.0 

In [67]:
n = FacetNormal(mesh)

M_n = inner(M, outer(n, n))

L_CDG = -inner(jump(theta, n), avg(M_n))*dS + \
           (1.0/2.0)*(alpha('+')/h_avg)*inner(jump(theta, n), jump(theta, n))*dS

In [68]:
class AllBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

In [69]:
all_boundary = AllBoundary()
bcs_w = [DirichletBC(W, Constant(0.0), all_boundary)]

In [70]:
facet_function = MeshFunction("size_t", mesh, mesh.geometry().dim() - 1)
facet_function.set_all(0)
all_boundary.mark(facet_function, 1)

In [71]:
ds = Measure("ds")(subdomain_data=facet_function)

In [72]:
theta_d = Constant((0.0, 0.0))

In [73]:
theta_effective = theta - theta_d 
L_BC = -inner(inner(theta_effective, n), M_n)*ds(1) + \
        (1.0/2.0)*(alpha/h)*inner(inner(theta_effective, n), inner(theta_effective, n))*ds(1) 

In [74]:
f = Constant(1)
W_ext = f*w_*dx

L = psi_M*dx - W_ext + L_CDG + L_BC

F = derivative(L, w_, w_t)
J = derivative(F, w_, w)

A, b = assemble_system(J, -F, bcs=bcs_w)
solver = PETScLUSolver("mumps")
solver.solve(A, w_.vector(), b)
XDMFFile("output/w.xdmf").write(w_)

In [75]:
def test_close():
    import numpy as np
    assert(np.isclose(w_((0.5, 0.5)), 1.265E-6, atol=1E-3, rtol=1E-3))

In [76]:
w_((0.5, 0.5))

1265.0722345646789